## Build Model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
#
# model.add(Dense(1))
# model.add(Activation('sigmoid'))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## Set Data Generator (If training)

In [ ]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
# train_generator = train_datagen.flow_from_directory(
#         'train',  # this is the target directory
#         target_size=(150, 150),  # all images will be resized to 150x150
#         batch_size=batch_size,
#         class_mode='binary')
train_generator = train_datagen.flow_from_directory(
        'train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')
# since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
# validation_generator = test_datagen.flow_from_directory(
#         'validation',
#         target_size=(150, 150),
#         batch_size=batch_size,
#         class_mode='binary')
validation_generator = test_datagen.flow_from_directory(
        'validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

## Fit the generator (if traning)

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2048 // batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=832 // batch_size)
model.save_weights('second_try.h5')  # always save your weights after training or during training

## Load weights (If NOT training)

In [ ]:
model.load_weights('second_try.h5')

## Predict image with this model

In [ ]:
import numpy as np
import wget

# predicting images
img_width, img_height = 150, 150
image_name = 'examples/example.jpg'
url = 'https://images.halloweencostumes.com/products/15085/1-1/adult-fluffy-dog-costume.jpg'
file_name = wget.download(url, image_name)
# Load the image and scale it to the input dimensions
img = load_img(image_name, target_size=(img_width, img_height))
#Helper function to load 
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
#Scale each pixel value as a float value within [0,1]
#Necessary since that is what is defined in the ImageDataGenerator
x = np.array(x,dtype="float")/255.0

classes = model.predict(x)

# print the classes, the images belong to
print(classes)